# Einführung

Im diesem Teil des Tutorials zeige ich dir die ersten Schritte mit LangChain. Dabei installieren wir die notwendigen Bibliotheken und schreiben den allerersten Basiscode um mit einem Sprachmodell wie ChatGPT (OpenAI) zu kommunizieren.


# Warum LangChain?

Die Anbieter von KIs (Sprachmodellen/LLMs) bieten häufig eigene APIs an, mit denen man mit die KI in die eigene Anwendung integrieren kann. So kann man z.B. mit der API von OpenAI bereits sehr einfach die ersten Aufrufe machen.
Damit wir uns nicht an die API der jeweiligen Anbieter binden müssen, bietet uns LangChain uns eine einheitliche API um mit allen möglichen LLMs zu kommunizieren. Je nach Anwendungsfall können wir sogar die verschiedenen LLMs in einer Anwendung verwenden. So können wir mit einem Open-Source-Modell sog. Embeddings erstellen (mehr dazu in einem anderen Kapitel) und die Chat-Funktion von ChatGPT (OpenAI) nutzen. Wir können auch jederzeit das verwendete LLM austauschen, ohne den Code neu schreiben zu müssen.
Zusätzlich dazu bietet LangChain vielfältige weitere Möglichkeiten. Mit den PromptTemplates können wir die Erstellung und Verwaltung von Prompts (Eingaben für die LLMs) bewerkstelligen, ohne irgendwelche Strings zusammen zu basteln. Wir können mit den Output Parsern die Ausgaben der LLMs maschinell auswerten und z.B. uns das Ergebnis als JSON zurück geben zu lassen, mit dem wir in unserem Programm arbeiten können. Spannend wird es spätestens dann, wenn wir die Agenten "erwecken". D.h. wir geben der KI Werkzeuge, mit denen die KI mehr als nur Texte erzeugen kann. Z.B. Informationen beschaffen, Berechnungen durchführen uvm.

# Installation notwendiger Bibliotheken

Für die meisten Beispiele dieses Tutorial brauchen wir einige Bibliotheken. Wir brauchen natürlich die LangChain-Bibliothek. Daneben brauchen wir die OpenAI Bibliothek um mit ChatGPT zu kommunizieren (direkt aber auch unter Verwendung von LangChain). Dann benötigen wir außerdem Python Dotenv damit wir den notwendigen API-Key aus einer .env Datei laden können. Das hat den Vorteil, dass wir den geheimen API-Key nicht direkt im Code hinterlegen müssen und statt dessen in der .env Datei speichern. Diese Datei wird nicht im Git-Repository eingecheckt (in .gitignore ist diese Datei hinterlegt) und wir somit nicht aus versehen unseren API-Key veröffentlichen.

Führe zunächst folgenden Befehl aus:

In [1]:
%pip install langchain openai python-dotenv tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/f4/2e/0adf6e264b996e263b1c57cad6560ffd5492a69beb9fd779ed0463d486bc/tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 20.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


# Beispiel: Direkte Verwendung der OpenAI-Bibliothek

Bevor wir mit LangChain starten, erstmal ein Beispiel dafür, wie man die OpenAI (ChatGPT) API direkt nutzen. Das ermöglicht uns die Verwendung zu vergleichen.

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [{"role": "user", "content": "Erkläre mir bitte, wie ich gute Eingaben für ChatGPT schreibe."}]

response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

print(response)

Wir sehen, dass wir mit wenigen Zeilen Code eine Anfrage an ChatGPT senden können. Dabei fallen ein paar Sachen auf. Das erste ist, dass wir eine Liste von Nachrichten (_messages_) an die API übergeben. Außerdem enthält die Antwort wesentlich mehr, als nur den reinen Text der Antwort.

Der Grund für die Liste der Nachrichten ist die, dass wir die Chat-Funktionalität nutzen. Der Server behält sich nicht die vorher eingegebenen Fragen und Antworten. D.h. man muss stets den gesamten Chat-Verlauf inkl. Antworten zurück übermitteln. Dadurch weiß die KI über welche Themen wir zuvor "gesprochen" haben. 
D.h. wenn ich jetzt eine Frage stellen möchte, die sich auf den bisherigen Chatverlauf bezieht, muss ich die Nachrichten-Liste erweitern:

In [ ]:
# Die Antwort der KI muss als Nachricht in die Liste hinten angehängt werden
messages.append(response["choices"][0]["message"]);

# Die neue Frage kommt noch dazu
messages.append({"role": "user", "content": "Danke für die Erklärung. Kannst du bitte die wesentlichen Punkte als kurze Liste zusammen fassen"})

# Erneuter Aufruf
response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

print(response)

In [ ]:
# Schönere Ausgabe 

from IPython.display import display, Markdown
display(Markdown(response["choices"][0]["message"]['content']))

Damit könnten wir schon eine Komplette Chat-Anwendung programmieren. Einfach Eingabe vom Benutzer holen, der Liste der Nachrichten hinzufügen, Aufruf ChatGPT, Antwort hinzufügen und das Ganze nochmal wiederholen.

Lasst uns jetzt erste Schritte mit LangChain machen. Wir werden später sehen, wo uns LangChain hilft, diese Schritte zu automatisieren und wesentlich bequemer und übersichtlicher zu gestalten.

In [9]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

llm = ChatOpenAI(model="gpt-3.5-turbo")

response = llm.predict("Spreche ich mit ChatGPT?")

print(response)

Ja, Sie sprechen mit ChatGPT. Wie kann ich Ihnen helfen?


Vom Code her, sind die Unterschiede zunächst nicht groß. Die Funktionen heißen etwas anders aber ansonsten wirkt das Ganze recht ähnlich. Der Unterschied ist jedoch, dass wir jederzeit das LLM austauschen können und anstelle von _ChatOpenAI_ eine andere LLM-Implementierung verwenden.

In den kommenden Kapiteln sehen wir dann, wie LangChain uns die Arbeit an vielen Stellen abnimmt und uns ermöglicht das Programm sauber und übersichtlich zu halten.